In [2]:
from pathlib import Path
import random
import pandas as pd

import IMP
import IMP.atom

import sys
sys.path.append("../../../src")
from weights import get_random_w_mat

#### incorrect minor or major conformation

In [9]:
native_0 = Path("/wynton/home/sali/mhancock/xray/dev/45_synthetic_native_4/data/pdbs/native_0.pdb")
native_1 = Path("/wynton/home/sali/mhancock/xray/dev/45_synthetic_native_4/data/pdbs/native_1.pdb")

In [10]:
decoy_df = pd.read_csv(Path(Path.home(), "Documents/xray/score_bench/data/268_decoys_1_state/rand1000.csv"), index_col=0)
decoy_df.head()

,pdb,ff,rmsd,w_0_0
0,/wynton/group/sali/mhancock/xray/sample_bench/...,2383.119589,0.071744,1.0
1,/wynton/group/sali/mhancock/xray/sample_bench/...,653.215513,0.143783,1.0
2,/wynton/group/sali/mhancock/xray/sample_bench/...,442.669839,0.181037,1.0
3,/wynton/group/sali/mhancock/xray/sample_bench/...,424.997480,0.117468,1.0
4,/wynton/group/sali/mhancock/xray/sample_bench/...,435.583240,0.181531,1.0


In [11]:
for i in range(len(decoy_df)):
    # new_pdb = str(native_0) + "," + str(decoy_df.loc[i,"pdb"])
    new_pdb = str(decoy_df.loc[i,"pdb"]) + "," + str(native_1)
    decoy_df.loc[i,"pdb"] = new_pdb

    decoy_df.loc[i, "w_0_0"] = 0.9
    decoy_df.loc[i, "w_1_0"] = 0.1

decoy_df.head()

,pdb,ff,rmsd,w_0_0,w_1_0
0,/wynton/group/sali/mhancock/xray/sample_bench/...,2383.119589,0.071744,0.9,0.1
1,/wynton/group/sali/mhancock/xray/sample_bench/...,653.215513,0.143783,0.9,0.1
2,/wynton/group/sali/mhancock/xray/sample_bench/...,442.669839,0.181037,0.9,0.1
3,/wynton/group/sali/mhancock/xray/sample_bench/...,424.997480,0.117468,0.9,0.1
4,/wynton/group/sali/mhancock/xray/sample_bench/...,435.583240,0.181531,0.9,0.1


In [12]:
new_decoy_file = Path(Path.home(), "Documents/xray/score_bench/data/268_decoys_1_state/rand1000_major.csv")
decoy_df.to_csv(new_decoy_file)

#### correct structure with wrong weights

In [46]:
decoy_df = pd.read_csv("../data/csvs/decoys.csv", index_col=0)
decoy_df["pdb"] = "/wynton/home/sali/mhancock/xray/dev/45_synthetic_native_4/data/pdbs/native.pdb"
decoy_df.to_csv("../data/csvs/decoys_w.csv")

#### wrong structure with correct weights

In [47]:
decoy_df = pd.read_csv("../data/csvs/decoys.csv", index_col=0)
decoy_df["w_0_0"] = .9
decoy_df["w_0_1"] = .1
decoy_df["w_1_0"] = .1
decoy_df["w_1_1"] = .9
decoy_df.to_csv("../data/csvs/decoys_c.csv")